<a href="https://colab.research.google.com/github/abaybektursun/OS-LLMs-Workshop/blob/master/OSS_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.7/993.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 6.7 MB/s eta 0:00:00


In [2]:
import torch

has_gpu = False
if torch.cuda.is_available():
    install_cmd = "CMAKE_ARGS=\"-DLLAMA_CUBLAS=on\" FORCE_CMAKE=1 pip install --upgrade --force-reinstall llama-cpp-python"
    has_gpu = True
    print("GPU is available")
else:
    install_cmd = "CMAKE_ARGS=\"-DLLAMA_OPENBLAS=on\" FORCE_CMAKE=1 pip install llama-cpp-python"
    print("GPU is not available, using CPU")


GPU is available


In [3]:
!{install_cmd}

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 10.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 42.5 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.1.57-cp310-cp310-linux_x86_64.whl size=252308 sha256=369a68b8eb6aeea3e54f7ad8f49cb3ceff20de1230c11d7ecca4e37dfa900757
  Stored in directory: /root/.cache/pip/wheels/87/9b/55/22559358a9af8074053b6c39406f5c06f0d391c052f77ca0c2
Successfully built llama-cpp-python
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uni

In [16]:
available_models = {
    "stable-vicuna-13B.ggmlv3.q4_0.bin":
         {
            "url":"https://huggingface.co/TheBloke/stable-vicuna-13B-GGML/resolve/main/stable-vicuna-13B.ggmlv3.q4_0.bin",
            "desc": "CarperAI StableVicuna" + \
                     "13 Billion Parameters, GGML, CPU Only, Q4"
         },
    "Wizard-Vicuna-13B-Uncensored.ggmlv3.q4_0.bin": 
         {
            "url":"https://huggingface.co/TheBloke/Wizard-Vicuna-13B-Uncensored-GGML/resolve/main/Wizard-Vicuna-13B-Uncensored.ggmlv3.q4_0.bin",
            "desc": "Wizard-Vicuna; 13 Billion Parameters, GGML, CPU ONLY, Q4"
         },
    "llama-7b.ggmlv3.q4_0.bin": 
         {
            "url":"https://huggingface.co/TheBloke/LLaMa-7B-GGML/resolve/main/llama-7b.ggmlv3.q4_0.bin",
            "desc": "Original LLaMa; 7 Billion Parameters, GGML, CPU/GPU, Q4"
         },
    "llama-deus-7b-v3.ggmlv3.q4_0.bin":
         {
            "url":"https://huggingface.co/TheBloke/llama-deus-7b-v3-GGML/resolve/main/llama-deus-7b-v3.ggmlv3.q4_0.bin",
            "desc": "LLaMA DEUS V3; 7 Billion Parameters, GGML, CPU/GPU, Q4"
         }
}

In [18]:
for model in available_models:
  print(model + ':')
  print("\t" + available_models[model]['desc'])

stable-vicuna-13B.ggmlv3.q4_0.bin:
	CarperAI StableVicuna13 Billion Parameters, GGML, CPU Only, Q4
Wizard-Vicuna-13B-Uncensored.ggmlv3.q4_0.bin:
	Wizard-Vicuna; 13 Billion Parameters, GGML, CPU ONLY, Q4
llama-7b.ggmlv3.q4_0.bin:
	Original LLaMa; 7 Billion Parameters, GGML, CPU/GPU, Q4
llama-deus-7b-v3.ggmlv3.q4_0.bin:
	LLaMA DEUS V3; 7 Billion Parameters, GGML, CPU/GPU, Q4


In [19]:
chosen_model = "llama-deus-7b-v3.ggmlv3.q4_0.bin"

In [20]:
!wget {available_models[chosen_model]["url"]}

--2023-06-06 22:06:10--  https://huggingface.co/TheBloke/llama-deus-7b-v3-GGML/resolve/main/llama-deus-7b-v3.ggmlv3.q4_0.bin
Resolving huggingface.co (huggingface.co)... 13.249.85.16, 13.249.85.92, 13.249.85.127, ...
Connecting to huggingface.co (huggingface.co)|13.249.85.16|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/57/0a/570a1297dec6fdfa2c36e42c33e01a303b00c63668ee56cb2f7482622a2b100d/2ac458de7cfcd73522f811d4742a72b6fe6578fed49cb288836ecadeea54e4cb?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-deus-7b-v3.ggmlv3.q4_0.bin%3B+filename%3D%22llama-deus-7b-v3.ggmlv3.q4_0.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1686345873&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzU3LzBhLzU3MGExMjk3ZGVjNmZkZmEyYzM2ZTQyYzMzZTAxYTMwM2IwMGM2MzY2OGVlNTZjYjJmNzQ4MjYyMmEyYjEwMGQvMmFjNDU4ZGU3Y2ZjZDczNTIyZjgxMWQ0NzQyYTcyYjZmZTY1NzhmZWQ0OWNiMj

In [21]:
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [22]:
template = """Question: {question}

Answer: Let's work this out in a step by step way to be sure we have the right answer."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [23]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# Verbose is required to pass to the callback manager

In [24]:
if has_gpu:
    llm = LlamaCpp(
        model_path=chosen_model,
        n_gpu_layers=1, n_batch=1,
        callback_manager=callback_manager, 
        verbose=True
    )
else:
    llm = LlamaCpp(
        model_path=chosen_model, 
        callback_manager=callback_manager,
        verbose=True
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [25]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [26]:
question = "What do to if I am attacked by a bear?"

llm_chain.run(question)

 Here are some steps you can take if you find yourself being attacked by a bear: 

1. Stay calm and don't panic, as this is what triggers most human reactions from animals.
2. Don’t make any sudden movements or loud noises that might startle the bear or cause it to feel threatened.
3. Make eye contact with the bear and show that you are not afraid by standing tall and firmly.
4. If possible, try to look calm and non-threatening while keeping a safe distance from the bear. 
5. If the bear charges at you, defend yourself using any means available like rocks, sticks, or your hands. Aim for the bear's face or throat, and hit it hard.
6. Once the bear has stopped moving, back away slowly while keeping an eye on the bear. Avoid turning your back on the bear and always keep it in sight. 
7. If the bear continues to approach you, throw something at it (such as rocks or logs) to distract it and give yourself time to run away.
8. Once you are safe, leave the area quickly but slowly so that the b

" Here are some steps you can take if you find yourself being attacked by a bear: \n\n1. Stay calm and don't panic, as this is what triggers most human reactions from animals.\n2. Don’t make any sudden movements or loud noises that might startle the bear or cause it to feel threatened.\n3. Make eye contact with the bear and show that you are not afraid by standing tall and firmly.\n4. If possible, try to look calm and non-threatening while keeping a safe distance from the bear. \n5. If the bear charges at you, defend yourself using any means available like rocks, sticks, or your hands. Aim for the bear's face or throat, and hit it hard.\n6. Once the bear has stopped moving, back away slowly while keeping an eye on the bear. Avoid turning your back on the bear and always keep it in sight. \n7. If the bear continues to approach you, throw something at it (such as rocks or logs) to distract it and give yourself time to run away.\n8. Once you are safe, leave the area quickly but slowly so 